In [1]:
import pandas as pd
import requests, os
from datetime import datetime
from obspy.geodetics.base import gps2dist_azimuth
import matplotlib.pyplot as plt
import time
import shutil
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows',None)

In [18]:
#read weather station information 找距地震測站15公里以內的氣象測站
## 氣象局沒有資料: 宜蘭古魯 花蓮豐里

rainsta_exc=pd.read_excel('rainfall_station.xlsx')
renddate=pd.to_datetime(rainsta_exc['撤站日期']).dt.strftime("%Y%m%d").fillna(99999999)
rainsta_exc=rainsta_exc[renddate.astype(int)>=20120101].reset_index()
rstacode=rainsta_exc['站號']
rstaname=rainsta_exc['站名']
rlon=rainsta_exc['經度']
rlat=rainsta_exc['緯度']
rcon=rainsta_exc['城市']

#read seismic station information
seissta_exc=pd.read_csv('/home/nancy/GMT/stn_info_hualien',header=None,sep=',')
sstacode=seissta_exc[0]
slon=seissta_exc[2]
slat=seissta_exc[1]

chosen_stacode=[]
chosen_staname=[]
chosen_lon=[]
chosen_lat=[]
chosen_con=[]
for i in range(len(sstacode)):
    #print(sstacode[i])
    for j in range(len(rstacode)):
        dis=gps2dist_azimuth(slat[i],slon[i],rlat[j],rlon[j])[0]
        if dis/1000 <= 15:
            chosen_stacode.append(str(rstacode[j]))
            chosen_staname.append(rstaname[j])
            chosen_lon.append(rlon[j])
            chosen_lat.append(rlat[j])
            chosen_con.append(rcon[j])
chosen_new=pd.DataFrame({'station code':chosen_stacode,'Country':chosen_con,'station name':chosen_staname,
                     'longitude':chosen_lon,'latitude':chosen_lat})
chosen_new=chosen_new.drop_duplicates().reset_index()
chosen_new=chosen_new.drop('index',axis=1)
chosen_new=chosen_new.sort_values(['Country'])

chosen=pd.read_csv('chosen_WeatherStation.csv',index_col=False)
chosen=chosen.append(chosen_new,ignore_index=True)
chosen=chosen.drop_duplicates(subset=['station code'])
chosen.to_csv('chosen_WeatherStation.csv',encoding='utf_8_sig',index=False)
## After, NEED get URL by self.

/tmp/ipykernel_2676183/1181805998.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  rainsta_exc=pd.read_excel('rainfall_station.xlsx')
/tmp/ipykernel_2676183/1181805998.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  chosen=chosen.append(chosen_new,ignore_index=True)


In [29]:
# #get date list
# def get_mon_list(startdate,enddate='2021-12-01'):
#     date_l=[datetime.strftime(x,'%Y-%m') for x in list(pd.date_range(start=startdate, end=enddate, freq='M'))]
#     return date_l
# mon_list=get_mon_list('2012-01-01')

shutil.copy('chosen_WeatherStation.csv','WeatherStation.csv')
csv=pd.read_csv('WeatherStation.csv',index_col=False)
rstacode=csv['station code']
URL=csv['URL']
startyear=2012

for ii in range(len(rstacode)):
    if not os.path.isdir(rstacode[ii]):
        os.mkdir(rstacode[ii])
    if not os.path.isfile('%s/%s_202112.csv' %(rstacode[ii],rstacode[ii])):
        print('Download data of %s' %rstacode[ii])
        for year in range(startyear,2022):
            for mon in range(1,13):
                url=URL[ii]+str(year)+'-'+str(mon).zfill(2)
                res=requests.get(url)
                df=pd.read_html(res.text)[1]
                csvname='%s/%s_%s%s.csv'%(rstacode[ii],rstacode[ii],year,str(mon).zfill(2))
                print(csvname)
                df.to_csv(csvname,encoding='utf_8_sig',index=False)
            time.sleep(5)
            break
        time.sleep(15)
        break


'WeatherStation.csv'

Download data of C1H910
C1H910/C1H910_201201.csv
C1H910/C1H910_201202.csv
C1H910/C1H910_201203.csv
C1H910/C1H910_201204.csv
C1H910/C1H910_201205.csv
C1H910/C1H910_201206.csv
C1H910/C1H910_201207.csv
C1H910/C1H910_201208.csv
C1H910/C1H910_201209.csv
C1H910/C1H910_201210.csv
C1H910/C1H910_201211.csv
C1H910/C1H910_201212.csv


In [ ]:
#weather station list
def get_area_stations():
    area=['https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466990&stname=%25E8%258A%25B1%25E8%2593%25AE&datepicker=2021-01#',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467060&stname=%25E8%2598%2587%25E6%25BE%25B3&datepicker=2021-01#',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467540&stname=%25E5%25A4%25A7%25E6%25AD%25A6&datepicker=2021-01#',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466910&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466880&stname=%25E6%259D%25BF%25E6%25A9%258B&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467490&stname=%25E8%2587%25BA%25E4%25B8%25AD&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467440&stname=%25E9%25AB%2598%25E9%259B%2584&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466940&stname=%25E5%259F%25BA%25E9%259A%2586&datepicker=2021-01',
         'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467050&stname=%25E6%2596%25B0%25E5%25B1%258B&datepicker=2021-01',]
    area_l=['花蓮縣','宜蘭縣','台東縣','臺北市','新北市','臺中市','臺南市','高雄市','基隆市','桃園市']
    sta_l=[]
    for url in area:
        stname.append(url.split('&')[-2].split('=')[-1])
        res=requests.get(url)
        df=pd.read_html(res.text)[0]
        df=df[2][0].split()

        cc=df.copy()
        for i in range(len(cc)):
            if cc[i]=='(撤銷站)':
                cc[i-1]=cc[i-1]+''+cc[i]
        stas=list(filter(lambda x: x!='(撤銷站)' , cc))
        sta_l.append(stas)
    station_l=pd.DataFrame({})
    return station_l

%25E8%258A%25B1%25E8%2593%25AE
%25E8%2598%2587%25E6%25BE%25B3
%25E5%25A4%25A7%25E6%25AD%25A6


In [47]:
df.columns

MultiIndex([('Unnamed: 0_level_0',          '觀測時間(day)',        'ObsTime'),
            (             'press',          '測站氣壓(hPa)',        'StnPres'),
            (             'press',         '海平面氣壓(hPa)',        'SeaPres'),
            (             'press',        '測站最高氣壓(hPa)',     'StnPresMax'),
            (             'press',      '測站最高氣壓時間(LST)', 'StnPresMaxTime'),
            (             'press',        '測站最低氣壓(hPa)',     'StnPresMin'),
            (             'press',      '測站最低氣壓時間(LST)', 'StnPresMinTime'),
            (       'temperature',              '氣溫(℃)',    'Temperature'),
            (       'temperature',            '最高氣溫(℃)',          'T Max'),
            (       'temperature',        '最高氣溫時間(LST)',     'T Max Time'),
            (       'temperature',            '最低氣溫(℃)',          'T Min'),
            (       'temperature',        '最低氣溫時間(LST)',     'T Min Time'),
            (         'Dew Point',            '露點溫度(℃)',   'Td dew point'),
            

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

r = requests.get('https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466910&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker=2019-08')
csvfile="test.csv"
r.encoding="utf-8"
soup=BeautifulSoup(r.text,"lxml")
tag_table=soup.find(id="MyTable")
rows=tag_table.findAll("tr")

with open(csvfile,"w+",newline="",encoding="utf-8") as fp:
    writer=csv.writer(fp)
    for row in rows:
        rowList=[]
        for cell in row.findAll(["td","th"]):
            rowList.append(cell.get_text().replace("\n","").replace("\r",""))
        writer.writerow(rowList)

In [16]:
cell.get_text().replace('\n','').replace('\r','')

'3.8\xa0'